# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

* Creating a `SparsePauliOp` from a list of strings

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.

In [1]:
import qiskit_alt
qiskit_alt.project.ensure_init()

We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
julia = qiskit_alt.project.julia
julia.Main.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into any of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
import qiskit_alt.electronic_structure
fermi_op = qiskit_alt.electronic_structure.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.electronic_structure.jordan_wigner(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

Note that the constant term differs. The qiskit-nature version ignores the nuclear-repulsion term. I need to open an issue about whether and how to handle it.

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, and record the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 4.01 ms
geometry=h2_geometry, basis='631g' 47.87 ms
geometry=h2_geometry, basis='631++g' 254.27 ms
geometry=h2o_geometry, basis='sto3g' 296.74 ms
geometry=h2o_geometry, basis='631g' 3851.07 ms


[4.01293010218069,
 47.87422340014018,
 254.26935179857537,
 296.73933099838905,
 3851.072468009079]

Next we benchmark qiskit_alt, and record the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.74 ms
geometry=h2_geometry, basis='631g' 3.68 ms
geometry=h2_geometry, basis='631++g' 19.85 ms
geometry=h2o_geometry, basis='sto3g' 16.94 ms
geometry=h2o_geometry, basis='631g' 474.94 ms


[0.7432029990013689,
 3.6772869993001223,
 19.853297900408506,
 16.941228799987584,
 474.9404695990961]

We compare the relative performance.

In [9]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[5.399507412608407,
 13.018897738809024,
 12.807411296303748,
 17.515809183723825,
 8.108537205220316]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [10]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 61.95 ms
geometry=h2_geometry, basis='631g' 102.37 ms
geometry=h2_geometry, basis='631++g' 298.09 ms
geometry=h2o_geometry, basis='sto3g' 449.94 ms
geometry=h2o_geometry, basis='631g' 25146.42 ms


[61.953555201762356,
 102.37395889998879,
 298.0937465967145,
 449.94441080489196,
 25146.419469994726]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [11]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 61.23 ms
geometry=h2_geometry, basis='631g' 64.01 ms
geometry=h2_geometry, basis='631++g' 75.70 ms
geometry=h2o_geometry, basis='sto3g' 78.40 ms
geometry=h2o_geometry, basis='631g' 185.11 ms


[61.22845840000082,
 64.00790859770495,
 75.69835009926464,
 78.39602120220661,
 185.11066940263845]

We compare the relative performance.

In [12]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[1.0118424801262271,
 1.5993954675728848,
 3.9379160339137993,
 5.739378145790738,
 135.84532729066078]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.ibm.com/John-Lapeyre/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.ibm.com/John-Lapeyre/ElectronicStructure.jl) provides an interface to electronic structure packages.

* [`ElectronicStructurePySCF.jl`](https://github.ibm.com/John-Lapeyre/ElectronicStructurePySCF.jl) provides an interface to `pyscf`

* [`QiskitQuantumInfo.jl`](https://github.ibm.com/John-Lapeyre/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* [Issue](https://github.ibm.com/John-Lapeyre/QuantumOps.jl/issues/6) for improving performance of Jordan-Wigner in `QuantumOps`.
    * Precompute one and two-body terms
    * Use @threads


# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [13]:
from qiskit_alt.pauli_operators import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 0.256429  +0.j        ,  0.60988298+0.j        ,  0.        +0.04315967j,
  0.02065799+0.j        ,  0.42571031+0.j        ,  0.41816184+0.j        ,
  0.        +0.22113221j, -0.12827795+0.j        ,  0.        +0.05537125j,
  0.        +0.04530526j, -0.05677472-0.j        ,  0.        +0.08304035j,
 -0.09362319+0.j        , -0.0152014 +0.j        ,  0.        +0.05176689j,
 -0.09027581+0.j        ,  0.58474571+0.j        ,  0.48025338+0.j        ,
  0.        +0.04944208j, -0.05206656+0.j        ,  0.38201

Note that the `numpy` matrix was automatically converted to a Julia `Matrix`.

We run benchmarks of conversion of matrices to the Pauli basis.

In [14]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.41 ms
nqubits=3, 5.03 ms
nqubits=4, 24.37 ms
nqubits=5, 140.40 ms
nqubits=6, 1693.99 ms
nqubits=7, 13543.24 ms
nqubits=8, 145778.53 ms


In [15]:
%run ../bench/from_matrix_alt.py

nqubits=2, 1.16 ms
nqubits=3, 0.77 ms
nqubits=4, 2.16 ms
nqubits=5, 5.03 ms
nqubits=6, 20.99 ms
nqubits=7, 99.51 ms
nqubits=8, 677.92 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [16]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[1.2176943679284769,
 6.52519135823632,
 11.271136775888447,
 27.911367044363708,
 80.6994760864295,
 136.09554371314735,
 215.03908651661197]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [17]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.3743672001291998 ms
k=10, n=100, 1.8459046492353082 ms
k=10, n=1000, 16.845469499821775 ms
k=10, n=5000, 83.12626869883388 ms
k=10, n=10000, 167.32282200246118 ms
k=10, n=100000, 1716.646151005989 ms
k=100, n=10, 0.40648595022503287 ms
k=100, n=100, 1.9967722502769902 ms
k=100, n=1000, 18.089473599684425 ms
k=100, n=5000, 88.40503570099827 ms
k=100, n=10000, 181.95561799802817 ms
k=100, n=100000, 1848.3988769876305 ms


Now, using `qiskit_alt`

In [18]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 0.6798120492021553 ms
k=10, n=100, 0.7528642992838286 ms
k=10, n=1000, 1.6406883500167169 ms
k=10, n=5000, 7.631268100521993 ms
k=10, n=10000, 13.870301948918495 ms
k=10, n=100000, 163.09510770079214 ms
k=100, n=10, 0.6631437499891035 ms
k=100, n=100, 0.8571260506869294 ms
k=100, n=1000, 4.54650714964373 ms
k=100, n=5000, 12.06824035034515 ms
k=100, n=10000, 24.05165149975801 ms
k=100, n=100000, 280.6339766495512 ms


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [19]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[0.5506922105434386,
 2.4518424515430572,
 10.267318287260428,
 10.892851306475249,
 12.063387128750128,
 10.525430070871778,
 0.6129680785375902,
 2.3296133032903508,
 3.978762818200327,
 7.325428822642722,
 7.565202663935942,
 6.58651136635485]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia takes all the time.

We see this in the following.

In [20]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


Generate `1000` ten-qubit Pauli strings.

In [21]:
%julia using Random: randstring
%julia pauli_strings = [randstring("IXYZ", 10) for _ in 1:1000]
None;

Benchmark converting these to a `QuantumOps.PauliSum`. Note that as the sums are always sorted.

In [22]:
%julia import Pkg; Pkg.add("BenchmarkTools")
%julia using BenchmarkTools: @btime
%julia @btime QuantumOps.PauliSum($pauli_strings)
None;

    Updating registry at `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.8.0-DEV/depot/registries/QuantumRegistry`
    Updating git-repo `https://github.com/Qiskit-Extensions/QuantumRegistry`
    Updating registry at `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.8.0-DEV/depot/registries/General.toml`
   Resolving package versions...
  No Changes to `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.8.0-DEV/Project.toml`
  No Changes to `~/julia_qiskit_repos/qiskit_alt/venv/julia_project/qiskit_alt-1.8.0-DEV/Manifest.toml`


  336.091 μs (1012 allocations: 134.17 KiB)


Check that we are actually getting the right result.

In [23]:
%julia pauli_sum = QuantumOps.PauliSum(pauli_strings);
%julia println(length(pauli_sum))
%julia println(pauli_sum[1])

1000
10-factor QuantumOps.PauliTerm{Vector{QuantumOps.Paulis.Pauli}, Complex{Int64}}:
IIIIYIXZYZ * (1 + 0im)


In [24]:
6.9 * 2.29 / .343  # Ratio of time to construct PauliSum via qiskit_alt to time in pure Julia

46.06705539358601

So the pure Julia code is `46` times faster than the qiskit.quantum_info.
**But, the `qiskit.quantum_info` is also extracting a possible phase !**